In [1]:
from pytket import Circuit
from pytket.circuit import OpType
from pytket.circuit.display import render_circuit_jupyter
from pytket_dqc.networks import NISQNetwork
from pytket_dqc.allocators import HypergraphPartitioning
from pytket_dqc.circuits import HypergraphCircuit
from pytket_dqc.packing import PacMan

In [2]:
circuit = Circuit(3)
circuit.add_gate(OpType.CU1,1,[0,1]).H(0).add_gate(OpType.CU1,1,[0,1]).H(0).H(0).add_gate(OpType.CU1,1,[0,1]).H(0).add_gate(OpType.CU1,0.5,[0,1]).H(0).add_gate(OpType.CU1,1,[0,1]).H(0).H(0).add_gate(OpType.CU1,1,[0,1]).H(0).add_gate(OpType.CU1,1.5,[0,1])
network = NISQNetwork(
    [[0,1],[0,2],[1,2]],
    {0: [0], 1: [1], 2: [2]}
)
render_circuit_jupyter(circuit)

In [5]:
hyper_circ = HypergraphCircuit(circuit)
distributor = HypergraphPartitioning()
placement = distributor.allocate(hyper_circ, network)

In [8]:
pacman = PacMan(hyper_circ, placement.placement)

Checking qubit vertex 0
Checking packing index 0
Checking 5 packets
Checking packet 0 against packet 2
Too many Hs
Checking packet 0 against packet 3
Checking packet 3 against packet 5
Too many Hs
Checking packet 3 against packet 6
Checking packing index 1
Checking 4 packets
Checking packet 1 against packet 3
Too many Hs
Checking packet 1 against packet 4
Too many Hs
Checking packet 1 against packet 5
Too many Hs
Checking packet 1 against packet 6
Too many Hs
Checking packing index 2
Checking 3 packets
Checking packet 2 against packet 4
Cannot embed CU1
Checking packet 2 against packet 5
Cannot embed CU1
Checking packet 2 against packet 6
Cannot embed CU1
Checking packing index 4
Checking 1 packets
Checking packet 4 against packet 6
Too many Hs
Checking qubit vertex 1
Checking qubit vertex 2
